## BigQuery ML experiment with consumer complaint dataset

### This experiment aims at evaluating logistic regression using Bigquery ML

### The dataset can be downloaded from https://catalog.data.gov/dataset/consumer-complaint-database

In [65]:
import pandas as pd
from google.cloud import bigquery
import numpy as np

In [73]:
consumer_complaints_df = pd.read_csv("Consumer_Complaints.csv",parse_dates=[0,13]).set_index("Date received")
consumer_complaints_df.head()

,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
Date received,,,,,,,,,,,,,,,,,
2014-03-12,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,M&T BANK CORPORATION,MI,48382,NaN,NaN,Referral,2014-03-17,Closed with explanation,Yes,No,759217
2017-01-19,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Received bad information about my loan,When my loan was switched over to Navient i wa...,NaN,"Navient Solutions, LLC.",LA,NaN,NaN,Consent provided,Web,2017-01-19,Closed with explanation,Yes,No,2296496
2018-04-06,Credit card or prepaid card,General-purpose credit card or charge card,"Other features, terms, or problems",Other problem,I tried to sign up for a spending monitoring p...,NaN,CAPITAL ONE FINANCIAL CORPORATION,VA,NaN,Older American,Consent provided,Web,2018-04-06,Closed with explanation,Yes,NaN,2866101
2014-06-08,Credit card,NaN,Bankruptcy,NaN,NaN,NaN,AMERICAN EXPRESS COMPANY,ID,83854,Older American,NaN,Web,2014-06-10,Closed with explanation,Yes,Yes,885638
2014-09-13,Debt collection,Credit card,Communication tactics,Frequent or repeated calls,NaN,NaN,"CITIBANK, N.A.",VA,23233,NaN,NaN,Web,2014-09-13,Closed with explanation,Yes,Yes,1027760


In [74]:
print(consumer_complaints_df.shape[0])
print (consumer_complaints_df.index.min())
print (consumer_complaints_df.index.max())
print(consumer_complaints_df.shape[0])

1164859
2011-12-01 00:00:00
2018-11-20 00:00:00
1164859


In [76]:
consumer_complaints_without_nan = consumer_complaints_df[consumer_complaints_df["Consumer complaint narrative"].notnull()]
print(consumer_complaints_without_nan.shape[0])
consumer_complaints_without_nan.head()

341828


,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
Date received,,,,,,,,,,,,,,,,,
2017-01-19,Student loan,Federal student loan servicing,Dealing with my lender or servicer,Received bad information about my loan,When my loan was switched over to Navient i wa...,NaN,"Navient Solutions, LLC.",LA,NaN,NaN,Consent provided,Web,2017-01-19,Closed with explanation,Yes,No,2296496
2018-04-06,Credit card or prepaid card,General-purpose credit card or charge card,"Other features, terms, or problems",Other problem,I tried to sign up for a spending monitoring p...,NaN,CAPITAL ONE FINANCIAL CORPORATION,VA,NaN,Older American,Consent provided,Web,2018-04-06,Closed with explanation,Yes,NaN,2866101
2018-08-22,Mortgage,Conventional home mortgage,Trouble during payment process,NaN,"My mortgage is with BB & T Bank, recently I ha...",Company has responded to the consumer and the ...,BB&T CORPORATION,IN,NaN,NaN,Consent provided,Web,2018-08-22,Closed with explanation,Yes,NaN,2998686
2016-11-14,Mortgage,Conventional fixed mortgage,Credit decision / Underwriting,NaN,The entire lending experience with Citizens Ba...,NaN,"CITIZENS FINANCIAL GROUP, INC.",NY,12054,NaN,Consent provided,Web,2016-11-14,Closed with explanation,Yes,No,2207648
2017-04-17,Credit reporting,NaN,Credit reporting company's investigation,Inadequate help over the phone,My credit score has gone down XXXX points in t...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,FL,32461,NaN,Consent provided,Web,2017-04-17,Closed with explanation,Yes,No,2437591


### Printing unique labels

In [91]:
consumer_complaints_without_nan["Product"].unique()

array(['Student loan', 'Credit card or prepaid card', 'Mortgage',
       'Credit reporting',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Debt collection', 'Vehicle loan or lease',
       'Money transfer, virtual currency, or money service',
       'Checking or savings account',
       'Payday loan, title loan, or personal loan', 'Consumer Loan',
       'Bank account or service', 'Credit card', 'Money transfers',
       'Prepaid card', 'Payday loan', 'Other financial service',
       'Virtual currency'], dtype=object)

### Number of records in training and test set

In [80]:
train_set = consumer_complaints_without_nan[( consumer_complaints_without_nan.index <= "2017-11-30 00:00:00" )]

print(train_set.shape[0])


238542


In [82]:
test_set = consumer_complaints_without_nan[consumer_complaints_without_nan.index > "2017-11-30 00:00:00"]
print(test_set.shape[0])

103286


## Loading data in Bigquery

In [37]:
client = bigquery.Client.from_service_account_json(
    '<path to service account.json file>')

In [38]:
dataset_id = 'bigquery_demo'
table_id = 'consumer_complaint'

In [39]:
dataset_ref = client.dataset(dataset_id)
dataset = client.get_dataset(dataset_ref)
table_ref = dataset_ref.table(table_id)

In [40]:
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

In [41]:
print('Dataset ID: {}'.format(dataset_id))
print('Description: {}'.format(dataset.description))

Dataset ID: bigquery_demo
Description: None


### Insert data in Bigquery from Pandas dataframe

In [30]:
job = client.load_table_from_dataframe(consumer_complaints_df, table_ref, location='EU')
job.result()

### Train a multi class logistic regression classifier

In [84]:
# Train a model
train_query = """
    create or replace model
    `bigquery_demo.consumer_complaint_model`
     options
      ( model_type='logistic_reg',
        auto_class_weights=true,
        input_label_cols=['Product'],
        max_iterations=10) AS
     select 
     Product, Consumer_complaint_narrative 
     from `bigquery_demo.consumer_complaint`
     where Consumer_complaint_narrative is not null
     and Date_received <= "2017-11-30 00:00:00"
     limit 100000
    """
training_job = client.query(train_query).to_dataframe()
print(training_job) 

Empty DataFrame
Columns: []
Index: []


### Get training info of the model

In [85]:
training_info = """
    select
      *
    from
      ML.TRAINING_INFO(MODEL `bigquery_demo.consumer_complaint_model`)
    """

training_info_job = client.query(training_info).to_dataframe()
print(training_info_job)

   training_run  iteration      loss  eval_loss  duration_ms  learning_rate
0             0          1  0.137454   0.160068       111464            0.4
1             0          0  0.151364   0.160474        95644            0.2


### Evaluate the trained model

In [86]:
# perform evaluation
query_evaluate = """
    select 
      * 
    from ML.EVALUATE (MODEL `bigquery_demo.consumer_complaint_model`, 
      (
        select 
        Product, Consumer_complaint_narrative 
        from `bigquery_demo.consumer_complaint`
        where Consumer_complaint_narrative is not null
        and Date_received > "2017-11-30 00:00:00"
      ))
    """

evaluation_job = client.query(query_evaluate).to_dataframe()
print(evaluation_job) 

   precision  recall  accuracy  f1_score  log_loss   roc_auc
0        NaN     NaN  0.223612       NaN  2.889783  0.000723


### Get predictions from the model

In [87]:
# perform prediction
query_predict = """
    select 
      *
    from ML.PREDICT (MODEL `bigquery_demo.consumer_complaint_model`,
      (
        select 
        Consumer_complaint_narrative 
        from `bigquery_demo.consumer_complaint`
        where Consumer_complaint_narrative is not null
        limit 2))
    """
prediction_job = client.query(query_predict).to_dataframe()
prediction_job.head() 

,predicted_Product,predicted_Product_probs,Consumer_complaint_narrative
0,Student loan,"[{'label': 'Student loan', 'prob': 0.062743874...",When my loan was switched over to Navient i wa...
1,Debt collection,"[{'label': 'Debt collection', 'prob': 0.055613...",I tried to sign up for a spending monitoring p...


### Probability distribution over all categories

In [89]:
prediction_job["predicted_Product_probs"][0]

[{'label': 'Student loan', 'prob': 0.06274387450241385},
 {'label': 'Debt collection', 'prob': 0.055192922390363434},
 {'label': 'Mortgage', 'prob': 0.055192589285376285},
 {'label': 'Credit reporting, credit repair services, or other personal consumer reports',
  'prob': 0.05519233083191825},
 {'label': 'Credit reporting', 'prob': 0.05519228276731627},
 {'label': 'Credit card', 'prob': 0.055191214614702826},
 {'label': 'Bank account or service', 'prob': 0.05519028700813079},
 {'label': 'Consumer Loan', 'prob': 0.05518841165685301},
 {'label': 'Credit card or prepaid card', 'prob': 0.05518574536003181},
 {'label': 'Checking or savings account', 'prob': 0.05517919845154978},
 {'label': 'Payday loan', 'prob': 0.05514652147326096},
 {'label': 'Vehicle loan or lease', 'prob': 0.055144268681504065},
 {'label': 'Prepaid card', 'prob': 0.055125338429545936},
 {'label': 'Money transfers', 'prob': 0.05512330302485571},
 {'label': 'Payday loan, title loan, or personal loan',
  'prob': 0.05511750

In [90]:
prediction_job["predicted_Product_probs"][1]

[{'label': 'Debt collection', 'prob': 0.055613002036629555},
 {'label': 'Mortgage', 'prob': 0.0556126663455983},
 {'label': 'Credit reporting, credit repair services, or other personal consumer reports',
  'prob': 0.05561240588556867},
 {'label': 'Credit reporting', 'prob': 0.05561235744779669},
 {'label': 'Credit card', 'prob': 0.05561128100149434},
 {'label': 'Student loan', 'prob': 0.05561057833076534},
 {'label': 'Bank account or service', 'prob': 0.05561034619152554},
 {'label': 'Consumer Loan', 'prob': 0.05560845627428762},
 {'label': 'Credit card or prepaid card', 'prob': 0.05560576926177735},
 {'label': 'Checking or savings account', 'prob': 0.05559917145541681},
 {'label': 'Payday loan', 'prob': 0.05556623975678285},
 {'label': 'Vehicle loan or lease', 'prob': 0.05556396936263171},
 {'label': 'Prepaid card', 'prob': 0.0555448909851409},
 {'label': 'Money transfers', 'prob': 0.055542839631215056},
 {'label': 'Payday loan, title loan, or personal loan',
  'prob': 0.0555369925320